<br/>

$$ \huge{\underline{\textbf{ Playing Atari Games with Deep RL }}} $$

$$ \large{\textbf{MountainCar + DQN + Memory Reply}} $$

<br/>



NOT TESTED

In [ ]:
def q_learning(env, frames, gamma, eps_decay_steps, eps_target, batch_size, model, mem, callback=None, trace=None):
    """Episodic Semi-Gradient Sarsa
    
    Params:
        env - environment
        ep - number of episodes to run
        gamma - discount factor [0..1]
        eps - epsilon-greedy param
        model      - function approximator, already initialised, with methods:
                     eval(state, action) -> float
                     train(state, target) -> None
    """
    def policy(st, model, eps):
        if np.random.rand() > eps:
            q_values = model.eval(np.array([st]))
            return np.argmax(q_values)
        else:
            return env.action_space.sample()
    
    if eps_decay_steps is not None:
        eps_delta = (1-eps_target) / eps_decay_steps
        eps = 1
    else:
        eps = eps_target
        
    # Fill memory buffer using random policy
    while len(mem) < mem.max_len:
        S = env.reset();
        for t_ in itertools.count():
            A = env.action_space.sample()    # random policy
            S_, R, done, _ = env.step(A)
            mem.append(S, A, R, S_, done)
            if done:
                break
            S = S_

    tts_ = 0                                 # total time step
    for e_ in itertools.count():             # count from 0 to infinity
        
        S = env.reset()
        
        for t_ in itertools.count():         # count from 0 to infinity
            
            A = policy(S, model, eps)
            
            S_, R, done, _ = env.step(A)
            
            mem.append(S, A, R, S_, done)
            
            if callback is not None:
                callback(e_, t_, S, A, R, done, eps, model, mem, trace)
            
            states, actions, rewards, n_states, dones, _ = mem.get_batch(batch_size)
            targets = model.eval(n_states)
            targets = rewards + gamma * np.max(targets, axis=-1)
            targets[dones] = rewards[dones]                # return of next-to-terminal state is just R
            model.train(states, actions, targets)

            if done:
                break
                
            if tts_ >= frames:
                return
                
            S = S_
            
            if eps > eps_target:
                eps = max(eps - eps_delta, eps_target)
                
            tts_ += 1

In [ ]:
def enjoy_env(env, frames, episodes, eps, model, callback=None, trace=None, render=True, sleep=0):

    def policy(st, model, eps):
        if np.random.rand() > eps:
            q_values = model.eval(np.array([st]))
            return np.argmax(q_values)
        else:
            return env.action_space.sample()
    
    tts_ = 0                                 # total time step
    for e_ in itertools.count():             # count from 0 to infinity
        
        S = env.reset()
        
        if render:
            env.render()
            time.sleep(sleep)
        
        for t_ in itertools.count():         # count from 0 to infinity
            
            A = policy(S, model, eps)
            
            S_, R, done, _ = env.step(A)
            
            if render:
                env.render()
                time.sleep(sleep)
            
            if callback is not None:
                callback(e_, t_, S, A, R, done, eps, model, None, trace)
    
            if done:
                break
                
            if tts_ >= frames:
                return
                
            S = S_
                
            tts_ += 1
            
        if e_ >= episodes-1:
            break

---

# Experiment Setup

NOT TESTED

Imports (source file: [tiles3.py](tiles3.py), [helpers_1001.py](helpers_1001.py))

In [ ]:
import pdb

In [ ]:
import time
import datetime
import numpy as np
import matplotlib.pyplot as plt
import tables
import itertools
import collections

import gym
import tensorflow as tf

Environment

In [ ]:
import helpers
import importlib
importlib.reload(helpers)

In [ ]:
import sys
sys.path.append('../Debug_NN')

In [ ]:
import importlib
import tables_logger
importlib.reload(tables_logger)

We will need callback to capture q-value array for whole state-action space at specified episodes.

In [ ]:
class Trace():
    def __init__(self, eval_every, render=False, test_states=None, test_labels=None, state_labels=None):
        if test_states is not None:
            assert test_states.ndim == 2
            assert len(test_states) == len(test_labels)
            
        self.eval_every = eval_every
        self.render = render
        self.test_states = test_states
        self.test_labels = test_labels
        self.state_labels = state_labels
        
        self.total_tstep = 0
        
        self.q_values = collections.OrderedDict()
        self.ep_end_idx = collections.OrderedDict()
        self.ep_rewards = collections.defaultdict(float)
        
        self.states = []
        self.actions = []
        self.rewards = []  # t+1
        self.dones = []    # t+1
        self.epsilons = []

In [ ]:
def callback(episode, tstep, st, act, rew_, done_, eps, model, memory, trace):
    """Called from gradient_MC after every episode.
    
    Params:
        episode [int] - episode number
        tstep [int]   - timestep within episode
        model [obj]   - function approximator
        trace [list]  - list to write results to"""
    
    if trace.total_tstep == 0 and memory is not None:
        print('num exits in memory:', np.count_nonzero(memory._hist_done_1))
        
    if done_:
        trace.ep_end_idx[episode] = trace.total_tstep
    
    trace.states.append(st)
    trace.actions.append(act)
    trace.rewards.append(rew_)
    trace.dones.append(done_)
    trace.epsilons.append(eps)
    
    trace.ep_rewards[episode] += rew_
    
    if trace.render:
        env.render()
            
    if trace.eval_every is not None:
        if trace.total_tstep % trace.eval_every == 0:
            print()
            print('■'*80)
            print('episode:', episode, '\t time step:', tstep,
                  '\t total time step:', trace.total_tstep, '\t eps:', round(eps,3), 
                  '\t wall time:', datetime.datetime.now())

            
        if trace.total_tstep % trace.eval_every == 0:
            
            if len(st) == 2:
                # We are working with 2D environment,
                # plot whole Q-Value functions across whole state space
            
                q_arr = helpers.eval_state_action_space(model, env, split=[128,128])
                trace.q_values[trace.total_tstep] = q_arr

                helpers.plot_mountain_car(env, episode, trace.total_tstep, 1000, trace, memory,
                                          axis_labels=['state[0]', 'state[1]'],
                                          action_labels=['Act 0', 'Act 1', 'Act 2'],
                                          action_colors=['red', 'blue', 'green'])
                
            else:
                # Environment is not 2D, so we can't plot whole Q-Value function
                # Instead we plot state on standard graph, which is still better than nothing
                
                if trace.test_states is not None:
                    y_hat = model.eval(trace.test_states)
                    trace.q_values[trace.total_tstep] = y_hat
                
                helpers.plot_generic_environment(env, trace.total_tstep, 1000, trace, memory)

    trace.total_tstep += 1

---

# Imports for testing

RUN FROM HERE

In [ ]:
import pdb

In [ ]:
import time
import datetime
import numpy as np
import matplotlib.pyplot as plt
import tables
import itertools
import collections

import gym
import tensorflow as tf

In [ ]:
import helpers
import importlib
importlib.reload(helpers)

In [ ]:
import sys
sys.path.append('../Debug_NN')

In [ ]:
import importlib
import tables_logger
importlib.reload(tables_logger)

# Pong Helpers

In [ ]:
for env in gym.envs.registry.all():
    if env.id.startswith('Pong'):
        print(env.id)

In [ ]:
from skimage.transform import resize
from skimage.color import rgb2gray

In [ ]:
def preprocess(obs):
    obs_rgb = rgb2gray(obs)
    obs_110x84 = resize(obs_rgb, output_shape=(110, 84), mode='reflect', anti_aliasing=True)
    obs_84x84 = obs_110x84[13:-13,:]
    obs_uint8 = (obs_84x84*255).astype(np.uint8)
    return obs_uint8

In [ ]:
def plot_frames(frames):
    stack = np.array(frames)  # convert LazyFrame to np.ndarray
    assert stack.shape == (84, 84, 4)
    fig, axes = plt.subplots(nrows=1, ncols=stack.shape[-1], figsize=[16,4])
    for i in range(stack.shape[-1]):
        axes[i].imshow(stack[:,:,i], cmap='gray', vmin=0, vmax=255)
        axes[i].set_title('frame '+str(i))
    plt.show()

In [ ]:
class LazyFrames:
    def __init__(self, frames):
        assert isinstance(frames, list)
        assert isinstance(frames[0], np.ndarray)
        self._frames = frames   # list of np.ndarray
        
    def __array__(self, dtype=None):
        # print('__ARRAY__ called')
        merged = np.stack(self._frames, axis=-1)
        if dtype is not None:
            merged = merged.astype(dtype)
        return merged
    
    def __str__(self):
        return str(np.round(np.stack(self._frames, axis=-1), decimals=4))

In [ ]:
class WrapAtari:
    def __init__(self, env):
        assert env.observation_space == gym.spaces.Box(low=0, high=255, shape=[210,160,3], dtype=np.uint8)
        assert env.action_space == gym.spaces.Discrete(6)
        
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=[84, 84, 4], dtype=np.uint8)
        self.action_space = env.action_space
        
        self._env = env
        self._frames = collections.deque(maxlen=4)
    
    def reset(self):
        raw_obs = self._env.reset()           # 160x120 RGB
        obs = preprocess(raw_obs)             # 84x84 grayscale
        for _ in range(self._frames.maxlen):
            self._frames.append(obs)          # replace all
        return LazyFrames(list(self._frames))
    
    def step(self, action):
        assert self.action_space.contains(action)
        raw_obs, rew, done, info = self._env.step(action)
        obs = preprocess(raw_obs)             # 84x84 grayscale
        self._frames.append(obs)
        return LazyFrames(list(self._frames)), np.sign(rew), done, info
    
    def render(self):
        self._env.render()
    
    def close(self):
        self._env.close()

# Enjoy Random Pong

THIS WORKS

In [ ]:
# Create, don't train
env = gym.make('PongDeterministic-v4')
env = WrapAtari(env)

In [ ]:
def callback_disp(episode, tstep, st, act, rew_, done_, eps, model, memory, trace):
    global axes
    if rew_ != 0:
        print('rew:', rew_)
    if done_:
        print('done:', done_)
    # plot_frames(st)

In [ ]:
# Enjoy agent
try:
    enjoy_env(env, frames=float('inf'), episodes=1, eps=1.0, model=None, callback=callback_disp)
except KeyboardInterrupt:
    pass
finally:
    env.close()

# Pong

NOT TESTED

In [ ]:
def experiment_pong(frames):
    
    tf_model = TFNeuralNet(nb_in=8, nb_hid_1=64, nb_hid_2=64, nb_out=6, lr=0.00025)
    model = TFFunctApprox(tf_model,
                          np.array([-1., -1., -1., -1., -1., -1., -1., -1.]),  # state space low
                          np.array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]),
                          rew_mean=0,
                          rew_std=1,
                          nb_actions=env.action_space.n)
    
    mem = Memory(max_len=100000, state_shape=(8,), state_dtype=float)
    
    trace = Trace(eval_every=1000,
                 test_states=np.array([[0, 1.4, 0, 0, 0, 0, 0, 0],     # init
                                       [0, 0.7, 0, 0, 0, 0, 0, 0],     # half way, no tilt
                                       [0, 0.0, 0, 0, 0, 0, 0, 0],]),  # landing pad
                 test_labels=['start', 'half-way', 'landing-pad'],
                 state_labels=['Pos.x', 'Pos.y', 'Vel.x', 'Vel.y', 'Angle', 'Ang. Vel', 'Left Leg', 'Right Leg'])
    
    if frames != 0:
        with tables.open_file('outarray.h5', mode='w') as f_:
            model._model.setup_logdb(f_)
            q_learning(env, frames=frames, gamma=.99, eps_decay_steps=50000, eps_target=0.1,
                       batch_size=4096, model=model, mem=mem, callback=callback, trace=trace)
    
    return trace, model, mem

In [ ]:
# Create, don't train
env = gym.make('PongDeterministic-v4')
trace_rl, model, mem = experiment_pong(frames=0)

In [ ]:
# Train from scratch
env = gym.make('PongDeterministic-v4')
trace_rl, model, mem = experiment_lunarlander_tf(frames=200000)

In [ ]:
# Save weights
model._model.save('./tf_models/PongNoFrameskip-v4.ckpt')

In [ ]:
# Load weights
model._model.load('./tf_models/PongNoFrameskip-v4.ckpt')

In [ ]:
def callback_disp(episode, tstep, st, act, rew_, done_, eps, model, memory, trace):
    if done_:
        print(rew_, done_)
    pdb.set_trace()

In [ ]:
# Enjoy agent
try:
    enjoy_env(env, frames=float('inf'), eps=1.0, model=model, callback=callback_disp)
except KeyboardInterrupt:
    pass
finally:
    env.close()

---

# Function Approximators and Memory

In [ ]:
class TFNeuralNet():
    def __init__(self, nb_out):
        
        self.nb_out = nb_out
        
        try:    sess.close()
        except: pass
        tf.reset_default_graph()
        
        self._log_filename = None
        self._dict_layers = {}

        self._x = tf.placeholder(name='x', shape=[None, 84, 84, 4], dtype=tf.float32)
        self._y = tf.placeholder(name='y', shape=[None, nb_out], dtype=tf.float32)

        model = tf.layers.conv2d(self._x, filters=16, kernel_size=[8, 8], strides=[4, 4],
                                 padding='valid', activation=tf.nn.relu, name='Conv_1')
            
        model = tf.layers.conv2d(model, filters=32, kernel_size=[4, 4], strides=[2, 2],
                                 padding='valid', activation=tf.nn.relu, name='Conv_2')
        
        model = tf.layers.flatten(model)
        model = tf.layers.dense(model, 256, activation=tf.nn.relu, name='Dense')
        self._y_hat = tf.layers.dense(model, nb_out, activation=None, name='Output')
        
        # self._mse = tf.reduce_mean( tf.pow(self._y - self._y_hat, 2) )
        self._loss = tf.losses.mean_squared_error(self._y, self._y_hat)

        # No gradient clipping
        #self._optimizer = tf.train.RMSPropOptimizer(learning_rate=0.00025, decay=0.0, momentum=0.95, epsilon=0.01)
        #self._grads_and_vars = self._optimizer.compute_gradients(self._loss)
        #self._train_op = self._optimizer.apply_gradients(self._grads_and_vars)
        
        # Global gradient clipping
#         self._optimizer = tf.train.RMSPropOptimizer(learning_rate=0.00025, decay=0.0, momentum=0.95, epsilon=0.01)
#         gradients, variables = zip(*self._optimizer.compute_gradients(self._loss))
#         gradients, _ = tf.clip_by_global_norm(gradients, 1)
#         self._train_op = self._optimizer.apply_gradients(zip(gradients, variables))
        
        # Per matrix
        self._optimizer = tf.train.RMSPropOptimizer(learning_rate=0.00025, decay=0.0, momentum=0.95, epsilon=0.01)
        gradients, variables = zip(*self._optimizer.compute_gradients(self._loss))
        gradients = [ None if gradient is None else tf.clip_by_norm(gradient, 1.0) for gradient in gradients ]
        self._train_op = self._optimizer.apply_gradients(zip(gradients, variables))
        
        

        self._sess = tf.Session()
        self._sess.run(tf.global_variables_initializer())
                
    def backward(self, x, y):
        assert x.ndim == 4
        assert y.ndim == 2
        assert x.shape == (32, 84, 84, 4)
        assert y.shape == (32, 6)
        
        summary, _, loss = self._sess.run([self._dict_layers, self._train_op, self._loss],
                                   feed_dict={self._x: x, self._y: y})
        
        if self._log_filename is not None:
            tables_logger.append_log(self._log_filename, summary)
        
        return loss
    
    def forward(self, x):
        return self._sess.run(self._y_hat, feed_dict={self._x: x})
    
    def save(self, filepath):
        saver = tf.train.Saver()
        saver.saver(self._sess, filepath)
        
    def load(self, filepath):
        saver = tf.train.Saver()
        saver.restore(self._sess, filepath)
        
    def setup_logdb(self, filename, batch_save):
        
        graph = tf.get_default_graph()

        dict_inout = {
            #'batch_x' : cnn._x[0:batch_save,:,:,:],
            'batch_y' : cnn._y[0:batch_save,:],
        }

        dict_conv_1 = {
            'W': graph.get_tensor_by_name('Conv_1/kernel:0'),
            'b': graph.get_tensor_by_name('Conv_1/bias:0'),
            'dW': graph.get_tensor_by_name('gradients/Conv_1/Conv2D_grad/tuple/control_dependency_1:0'),
            'db': graph.get_tensor_by_name('gradients/Conv_1/BiasAdd_grad/tuple/control_dependency_1:0'),
            'z': graph.get_tensor_by_name('Conv_1/BiasAdd:0')[0:batch_save,:,:,:],
        }

        dict_conv_2 = {
            'W': graph.get_tensor_by_name('Conv_2/kernel:0'),
            'b': graph.get_tensor_by_name('Conv_2/bias:0'),
            'dW': graph.get_tensor_by_name('gradients/Conv_2/Conv2D_grad/tuple/control_dependency_1:0'),
            'db': graph.get_tensor_by_name('gradients/Conv_2/BiasAdd_grad/tuple/control_dependency_1:0'),
            'z': graph.get_tensor_by_name('Conv_2/BiasAdd:0')[0:batch_save,:,:,:],
        }

        dict_dense = {
            'W': graph.get_tensor_by_name('Dense/kernel:0')[:100,:50],
            'b': graph.get_tensor_by_name('Dense/bias:0'),
            'dW': graph.get_tensor_by_name('gradients/Dense/MatMul_grad/tuple/control_dependency_1:0')[:100,:50],
            'db': graph.get_tensor_by_name('gradients/Dense/BiasAdd_grad/tuple/control_dependency_1:0'),
            'z': graph.get_tensor_by_name('Dense/BiasAdd:0')[0:batch_save,:],
        }

        dict_output = {
            'W': graph.get_tensor_by_name('Output/kernel:0'),
            'b': graph.get_tensor_by_name('Output/bias:0'),
            'dW': graph.get_tensor_by_name('gradients/Output/MatMul_grad/tuple/control_dependency_1:0'),
            'db': graph.get_tensor_by_name('gradients/Output/BiasAdd_grad/tuple/control_dependency_1:0'),
            'z': graph.get_tensor_by_name('Output/BiasAdd:0')[0:batch_save,:],
        }

        dict_metrics = {
            'loss': cnn._loss,
        }

        self._log_filename = filename
        self._dict_layers = {
            'inout': dict_inout,
            'conv_1': dict_conv_1,
            'conv_2': dict_conv_2,
            'dense': dict_dense,
            'output': dict_output,
            'metrics': dict_metrics,
        }

        tables_logger.create_log(filename, self._dict_layers, batch_save)

In [ ]:
class TFFunctApprox():

    def __init__(self, model, st_low, st_high, rew_mean, rew_std, nb_actions):
        """Q-function approximator using Keras model

        Args:
            model: Keras compiled model
        """
        self._model = model
        
        assert np.isscalar(st_low) and np.isscalar(st_high)
        
        if nb_actions != model.nb_out:
            raise ValueError('Output shape does not match action_space shape')

        # normalise inputs
        self._offsets = st_low + (st_high - st_low) / 2
        self._scales = 1 / ((st_high - st_low) / 2)
        
        self._rew_mean = rew_mean
        self._rew_std = rew_std

    def eval(self, states):
        assert isinstance(states, np.ndarray)
        assert states.ndim == 4
        assert states.shape == (32, 84, 84, 4)
        
        inputs = (states - self._offsets) * self._scales

        y_hat = self._model.forward(inputs)
        
        return y_hat*self._rew_std + self._rew_mean

    def train(self, states, actions, targets):
        
        assert isinstance(states, np.ndarray)
        assert isinstance(actions, np.ndarray)
        assert isinstance(targets, np.ndarray)
        assert states.ndim == 4
        assert actions.ndim == 1
        assert targets.ndim == 1
        assert len(states) == len(actions) == len(targets)
        
        targets = (targets-self._rew_mean) / self._rew_std    # decreases range (std>1) to approx -1..1

        inputs = (states - self._offsets) * self._scales
        all_targets = self._model.forward(inputs)             # this range should be small already
        all_targets[np.arange(len(all_targets)), actions] = targets
        return self._model.backward(inputs, all_targets)


In [ ]:
class Memory:
    """Circular buffer for DQN memory reply. Fairly fast."""

    def __init__(self, max_len, state_shape, state_dtype):
        """
        Args:
            max_len: maximum capacity
        """
        assert isinstance(max_len, int)
        assert max_len > 0

        self.max_len = max_len                            # maximum length        
        self._curr_insert_ptr = 0                          # index to insert next data sample
        self._curr_len = 0                                 # number of currently stored elements

        state_arr_shape = [max_len] + list(state_shape)

        self._hist_St = np.zeros(state_arr_shape, dtype=state_dtype)
        self._hist_At = np.zeros(max_len, dtype=int)
        self._hist_Rt_1 = np.zeros(max_len, dtype=float)
        self._hist_St_1 = np.zeros(state_arr_shape, dtype=state_dtype)
        self._hist_done_1 = np.zeros(max_len, dtype=bool)

    def append(self, St, At, Rt_1, St_1, done_1):
        """Add one sample to memory, override oldest if max_len reached.

        Args:
            St [np.ndarray]   - state
            At [int]          - action
            Rt_1 [float]      - reward
            St_1 [np.ndarray] - next state
            done_1 [bool]       - next state terminal?
        """
        self._hist_St[self._curr_insert_ptr] = St
        self._hist_At[self._curr_insert_ptr] = At
        self._hist_Rt_1[self._curr_insert_ptr] = Rt_1
        self._hist_St_1[self._curr_insert_ptr] = St_1
        self._hist_done_1[self._curr_insert_ptr] = done_1
        
        if self._curr_len < self.max_len:                 # keep track of current length
            self._curr_len += 1
            
        self._curr_insert_ptr += 1                         # increment insertion pointer
        if self._curr_insert_ptr >= self.max_len:         # roll to zero if needed
            self._curr_insert_ptr = 0

    def __len__(self):
        """Number of samples in memory, 0 <= length <= max_len"""
        return self._curr_len

    def get_batch(self, batch_len):
        """Sample batch of data, with repetition

        Args:
            batch_len: nb of samples to pick

        Returns:
            states, actions, rewards, next_states, next_done, indices
            Each returned element is np.ndarray with length == batch_len
        """
        assert self._curr_len > 0
        assert batch_len > 0

        
        indices = np.random.randint(                   # randint much faster than np.random.sample
            low=0, high=self._curr_len, size=batch_len, dtype=int)

        states = np.take(self._hist_St, indices, axis=0)
        actions = np.take(self._hist_At, indices, axis=0)
        rewards_1 = np.take(self._hist_Rt_1, indices, axis=0)
        states_1 = np.take(self._hist_St_1, indices, axis=0)
        dones_1 = np.take(self._hist_done_1, indices, axis=0)
        
        if states.dtype == object and isinstance(mem._hist_St[0], LazyFrames): 
            states = np.stack(states)       # convert to single np.ndarray shape [batch_size, 4, 84, 84]
            states_1 = np.stack(states_1)   # where '4' is number of history frames presented to agent

        return states, actions, rewards_1, states_1, dones_1, indices


    
    def pick_last(self, nb):
        """Pick last nb elements from memory
        
        Returns:
            states, actions, rewards, next_states, done_1, indices
            Each returned element is np.ndarray with length == batch_len
        """
        assert nb <= self._curr_len
        
        start = self._curr_insert_ptr - nb                # inclusive
        end = self._curr_insert_ptr                       # not inclusive
        indices = np.array(range(start,end), dtype=int)   # indices to pick, can be negative
        indices[indices < 0] += self._curr_len            # loop negative to positive
        
        states = np.take(self._hist_St, indices, axis=0)
        actions = np.take(self._hist_At, indices, axis=0)
        rewards_1 = np.take(self._hist_Rt_1, indices, axis=0)
        states_1 = np.take(self._hist_St_1, indices, axis=0)
        dones_1 = np.take(self._hist_done_1, indices, axis=0)
        
        if states.dtype == object and isinstance(mem._hist_St[0], LazyFrames): 
            states = np.stack(states)       # convert to single np.ndarray shape [batch_size, 4, 84, 84]
            states_1 = np.stack(states_1)   # where '4' is number of history frames presented to agent
        
        return states, actions, rewards_1, states_1, dones_1, indices
    

# Pong NN Test

In [ ]:
def mem_fill(env, mem, one_episode=False):
        
    # Fill memory buffer using random policy
    while len(mem) < mem.max_len:
        S = env.reset();
        for t_ in itertools.count():
            A = env.action_space.sample()    # random policy
            S_, R, done, _ = env.step(A)
            mem.append(S, A, R, S_, done)
            if done:
                if one_episode:
                    return
                break
            S = S_

In [ ]:
env = gym.make('PongDeterministic-v4')
env = WrapAtari(env)

In [ ]:
mem = Memory(max_len=10000, state_shape=(), state_dtype=object)
mem_fill(env, mem, one_episode=False)
print(len(mem))

In [ ]:
states, actions, rewards, n_states, dones, _ = mem.pick_last(len(mem))

In [ ]:
np.count_nonzero(rewards==-1)

In [ ]:
np.count_nonzero(rewards==1)

In [ ]:
np.count_nonzero(rewards==0)

In [ ]:
del states
del n_states

In [ ]:
cnn = TFNeuralNet(nb_out=6)
#cnn.setup_logdb('outarray.h5', 5)

In [ ]:
model = TFFunctApprox(cnn, st_low=0, st_high=255, rew_mean=0, rew_std=1, nb_actions=6)

In [ ]:
gamma = 0.95

In [ ]:
batch_size = 32

THIS SHOULD CONVERGE

In [ ]:
losses = []
for i in range(50000):
    states, actions, rewards, n_states, dones, _ = mem.get_batch(batch_size)
    targets = model.eval(n_states)
    targets = rewards + gamma * np.max(targets, axis=-1)
    targets[dones] = rewards[dones]                # return of next-to-terminal state is just R
    loss = model.train(states, actions, targets)
    
    losses.append(loss)
    if i % 25 == 0:
        print(i, loss)

# Test CNN Forward

In [ ]:
env = gym.make('PongDeterministic-v4')
env = WrapAtari(env)

In [ ]:
mem = Memory(max_len=1000, state_shape=(), state_dtype=object)

In [ ]:
mem_fill(env, mem)

In [ ]:
states, actions, rewards_1, states_1, dones_1, indices = mem.get_batch(10)

In [ ]:
for i in range(10):
    print('----')
    print(rewards_1[i])
    plot_frames(states[i])

In [ ]:
cnn = TFNeuralNet(nb_out=6)

In [ ]:
states, actions, rewards_1, states_1, dones_1, indices = mem.get_batch(10)

In [ ]:
states_nn = states / 255

In [ ]:
cnn.forward(states_nn)

In [ ]:
tf.trainable_variables()

In [ ]:
writer = tf.summary.FileWriter(logdir='tf_log', graph=cnn._sess.graph)
writer.flush()

# Test CNN Logging

In [ ]:
filename = 'outarray.h5'

In [ ]:
cnn = TFNeuralNet(nb_out=6)

In [ ]:
cnn.setup_logdb(filename, batch_save=10)

In [ ]:
tables_logger.print_log(filename)

# Test Lazy Frame

In [ ]:
A = np.array([1, 1, 1])
B = np.array([2, 2, 2])
C = np.array([3, 3, 3])

lf1 = LazyFrame([A, B])
lf2 = LazyFrame([B, C])

In [ ]:
mem = np.zeros(shape=[10], dtype=object)

In [ ]:
mem[0] = lf1
mem[1] = lf2

In [ ]:
lf1._frames[0][0] = 4

In [ ]:
lf1._frames

In [ ]:
np.stack(mem[[0,1]])

In [ ]:
np.array(mem[0])

# Test Mem Object

In [ ]:
env = gym.make('PongDeterministic-v4')
env = WrapAtari(env)

In [ ]:
lframes = env.reset()

In [ ]:
lframes_, rew_, done_, _ = env.step(0)

In [ ]:
mem = Memory(10, (), object)

In [ ]:
mem.append(lframes, 0, rew_, lframes_, done_)

In [ ]:
lframes = lframes_
lframes_, rew_, done_, _ = env.step(0)
mem.append(lframes, 0, rew_, lframes_, done_)

In [ ]:
print(mem._hist_St)
print(mem._hist_At)
print(mem._hist_Rt_1)
print(mem._hist_St_1)
print(mem._hist_done_1)

In [ ]:
arr = np.take(mem._hist_St, np.array([0, 1]), axis=0)

In [ ]:
arr

In [ ]:
np.stack(arr).shape

In [ ]:
plot_frames(np.stack(arr)[0])

In [ ]:
plot_frames(np.stack(arr)[1])

In [ ]:
states, actions, rewards_1, states_1, dones_1, indices = mem.get_batch(2)

In [ ]:
print(states.shape)
print(actions.shape)
print(rewards_1.shape)
print(states_1.shape)
print(dones_1.shape)